# Set Fixed Psats

Read default yaml files, write out fixed Psats according to a prescription.

LATS:  CHLAT and SPLAT use average value.
SPLAT:  MF1_1 and MF2_1 averaged, same for MF1_2 and MF2_2

In [22]:
# See what yaml files are locally available.
!ls *.yaml

CHLAT_20210605.yaml             SAT_pole_hdpeLFMF_20210622.yaml
SAT_pole_20210605.yaml          SPLAT_20210605.yaml
SAT_pole_SiHF_20210622.yaml


In [23]:
# Standard imports
import numpy as np
import yaml
import toml
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 20]

# bolo-calc import
from bolo import Top

In [42]:
data = {}
fixed_psats = {}
yamlfiles = {'CHLAT':'CHLAT_20210605',
             'SPLAT':'SPLAT_20210605',
             'SAT1' :'SAT_pole_hdpeLFMF_20210622',
             'SAT2' :'SAT_pole_SiHF_20210622'}

print('Psats derived from optical power and Psat safety factor')
for tel in yamlfiles.keys(): 
    yamlfile = yamlfiles[tel]+'.yaml'
    fixed_psats[yamlfiles[tel]]={}
    dd = yaml.safe_load(open(yamlfile))
    version_date = str(dd['version']['date'])
    telescope = str(dd['version']['name'])
    data[telescope] = {}
    print('{0:8s}  Psat(pW)'.format(telescope))
    del dd['version']

    # location of atmosphere file
    dd['sim_config']['config_dir'] = '../../bolo-calc/config'

    top = Top(**dd)
    top.run()
    
    for chan in top.instrument.tables.keys():
        if chan.split('_')[0] == 'cam' :
            chname = chan.split('_')[2] + '_' + chan.split('_')[3]
            data[telescope][chname] = {}
            #
            # Grab things we're going to print and give them short names and store them in the dictionary.
            psat = top.instrument.tables[chan]['P_sat'][0]
            data[telescope][chname]['psat'] = psat
            
            print('{0:5s} : {1:8.2f}'.format(chname,psat))
    print(' ')


# how many decimals to round to.
ddd = 3

print('===========================================================')
print('LAT Psats (Average of CHLAT and SPLAT for common bands)')
chan = 'ULF_1'
fixed_psats[yamlfiles['SPLAT']][chan]= np.round(data['SPLAT'][chan]['psat'],ddd)
print('{0:5s} : {1:8.3f}'.format(chan,data['SPLAT'][chan]['psat']))
for chan in data['CHLAT'].keys():
    psat_avg = np.round((data['CHLAT'][chan]['psat'] + data['SPLAT'][chan]['psat'])/2,ddd)
    fixed_psats[yamlfiles['CHLAT']][chan]= psat_avg
    fixed_psats[yamlfiles['SPLAT']][chan]= psat_avg
    print('{0:5s} : {1:8.3f}'.format(chan,psat_avg))
    

print('======================')
print('SAT Psats (Average of _1 and _2 in MF bands)')

fixed_psats[yamlfiles['SAT1']]['LF_1']= np.round(data['SAT_poleLFMF']['LF_1']['psat'],ddd)
fixed_psats[yamlfiles['SAT1']]['LF_2']= np.round(data['SAT_poleLFMF']['LF_2']['psat'],ddd)

avg90 = np.round((data['SAT_poleLFMF']['MF1_1']['psat'] + data['SAT_poleLFMF']['MF2_1']['psat'])/2,ddd)
fixed_psats[yamlfiles['SAT1']]['MF1_1']= avg90
fixed_psats[yamlfiles['SAT1']]['MF2_1']= avg90
#
avg150 = np.round((data['SAT_poleLFMF']['MF1_2']['psat'] + data['SAT_poleLFMF']['MF2_2']['psat'])/2,ddd)
fixed_psats[yamlfiles['SAT1']]['MF1_2']= avg150
fixed_psats[yamlfiles['SAT1']]['MF2_2']= avg150

fixed_psats[yamlfiles['SAT2']]['HF_1']= np.round(data['SAT_poleHF']['HF_1']['psat'],ddd)
fixed_psats[yamlfiles['SAT2']]['HF_2']= np.round(data['SAT_poleHF']['HF_2']['psat'],ddd)

for chan in fixed_psats[yamlfiles['SAT1']].keys():
    print('{0:5s} : {1:8.3f}'.format(chan,fixed_psats[yamlfiles['SAT1']][chan]))     
for chan in fixed_psats[yamlfiles['SAT2']].keys():
    print('{0:5s} : {1:8.3f}'.format(chan,fixed_psats[yamlfiles['SAT2']][chan]))     

    

Psats derived from optical power and Psat safety factor
CHLAT     Psat(pW)
LF_1  :     0.69
LF_2  :     3.63
MF_1  :     4.14
MF_2  :    12.88
HF_1  :    37.51
HF_2  :    55.28
 
SPLAT     Psat(pW)
ULF_1 :     0.41
LF_1  :     0.69
LF_2  :     4.30
MF_1  :     4.66
MF_2  :    11.52
HF_1  :    29.10
HF_2  :    39.99
 
SAT_poleLFMF  Psat(pW)
LF_1  :     1.40
LF_2  :     6.06
MF1_1 :     7.28
MF1_2 :    12.00
MF2_1 :     7.54
MF2_2 :    13.06
 
SAT_poleHF  Psat(pW)
HF_1  :    29.63
HF_2  :    38.62
 
LAT Psats (Average of CHLAT and SPLAT for common bands)
ULF_1 :    0.406
LF_1  :    0.686
LF_2  :    3.965
MF_1  :    4.401
MF_2  :   12.199
HF_1  :   33.308
HF_2  :   47.636
SAT Psats (Average of _1 and _2 in MF bands)
LF_1  :    1.396
LF_2  :    6.061
MF1_1 :    7.410
MF2_1 :    7.410
MF1_2 :   12.530
MF2_2 :   12.530
HF_1  :   29.634
HF_2  :   38.622


In [29]:
fixed_psats

{'CHLAT_20210605': {'LF_1': 0.686,
  'LF_2': 3.965,
  'MF_1': 4.401,
  'MF_2': 12.199,
  'HF_1': 33.308,
  'HF_2': 47.636},
 'SPLAT_20210605': {'ULF_1': 0.406,
  'LF_1': 0.686,
  'LF_2': 3.965,
  'MF_1': 4.401,
  'MF_2': 12.199,
  'HF_1': 33.308,
  'HF_2': 47.636},
 'SAT_pole_hdpeLFMF_20210622': {},
 'SAT_pole_SiHF_20210622': {}}

In [28]:
data

{'CHLAT': {'LF_1': {'psat': 0.6855981527506452},
  'LF_2': {'psat': 3.625274540618906},
  'MF_1': {'psat': 4.138045579296639},
  'MF_2': {'psat': 12.881304018415463},
  'HF_1': {'psat': 37.51364630268002},
  'HF_2': {'psat': 55.278270250645676}},
 'SPLAT': {'ULF_1': {'psat': 0.40607971104307855},
  'LF_1': {'psat': 0.6867501820413459},
  'LF_2': {'psat': 4.304658120263335},
  'MF_1': {'psat': 4.663977011750589},
  'MF_2': {'psat': 11.516984869436003},
  'HF_1': {'psat': 29.10236707927759},
  'HF_2': {'psat': 39.99444966747928}},
 'SAT_poleLFMF': {'LF_1': {'psat': 1.2392443657615442},
  'LF_2': {'psat': 5.8084811283747095},
  'MF1_1': {'psat': 6.640926497868591},
  'MF1_2': {'psat': 10.491718160831722},
  'MF2_1': {'psat': 6.8090642509402795},
  'MF2_2': {'psat': 11.339276648946933}},
 'SAT_poleHF': {'HF_1': {'psat': 26.479359328372695},
  'HF_2': {'psat': 34.995515852063335}}}

# Save fixed psats to toml file.

In [10]:
# Write to toml file
with open('FixedPsats.toml', "w") as toml_file:
    toml.dump(fixed_psats, toml_file, encoder=toml.TomlNumpyEncoder())